# Лекция 2. Углубление в НС и библиотеку Keras


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Introduction

На прошлой лекции мы познакомились с базовым набором инструментов библиотеки Keras, научились строить модели с помощью инструмента ```keras.Sequential()```. Однако на практике не все модели удается реализовать с помощью этого метода.

Если в модели присутствует "нелинейная" структура (т.е. есть разветвление потока данных), то ```keras.Sequential()``` не подойдет для построения такой модели. На помощь приходит инструмент *functional API*.


Рассмотрим следующую модель:

<div class="k-default-codeblock">
```
(input: 784-dimensional vectors)
       ↧
[Dense (32 units, relu activation)]
       ↧
[Dense (32 units, relu activation)]
       ↧
[Dense (10 units, softmax activation)]
       ↧
(output: logits of a probability distribution over 10 classes)
```
</div>

Обычно в глубоком обучении принято использовать следующую структуру во входных данных:

$$[n, N_{ch}, W,H],$$

где $n$ - размер подвыборки (batch_size), $N_{ch}$ - кол-во каналов (красный, зеленый, синий), $W$, $H$ - ширина и высота изображения. Однако batch_size не указывается при объявлении слоев в Keras. 



In [2]:
img_inputs = keras.Input(shape=(32, 32, 3))
print(img_inputs.shape)

(None, 32, 32, 3)


На сегодняшний день мы пока забудем о вышеописанной структуре и будем все входные данные трактовать как один длинный вектор длины $(N_{ch}\cdot W \cdot H)$.

In [3]:
inputs = keras.Input(shape=(784,))

In [4]:
print("Shape of input is", inputs.shape)
print("Data type of input is", inputs.dtype)

Shape of input is (None, 784)
Data type of input is <dtype: 'float32'>


Следующий слой, который будет использовать тензор `inputs` как входные данные, должен быть объявлен следующим образом:


In [4]:
x = layers.Dense(32)(inputs) ### объявление слоя
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Activation('relu')(x)
#x = dense1(inputs) ### указание входного тензора

y = layers.Dense(32)(inputs)
y = layers.BatchNormalization()(y)
y = layers.Dropout(0.5)(y)
y = layers.Activation('relu')(y)

dense21 = layers.Dense(32, activation="relu")(x)
dense22 = layers.Dense(32, activation='relu')(y)

dense3 = layers.Add()([dense21, dense22])

In [5]:
x = layers.Dense(32, activation="relu")(dense3)
outputs = layers.Dense(10)(x)

На этом этапе мы можем закончить добавление новых блоков и объявить модель, которая будет состоять из данных слоёв:

In [6]:
model = keras.Model(inputs=inputs, outputs=outputs, name="our_first_model")

In [7]:
model.summary()

Model: "our_first_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 32)           25120       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 32)           25120       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32)          128         ['dense[0][0]']                  
 alization)                                                                         

Можно, например, нарисовать модель в виде направленного графа:

In [8]:
keras.utils.plot_model(model, "my_first_model.png")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [9]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


## Обучение модели

Обучение модели происходит тем же самым образом, что и модели, объявленной с помощью класса ```Sequential()```, т.е. с помощью метода ```fit()```.


In [11]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 14s 1us/step


In [12]:
x_train = x_train.reshape(60000, 784).astype("float32") / 255.
x_test = x_test.reshape(10000, 784).astype("float32") / 255.

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), ### ОТЛИЧИЕ_ОТ_CategoricalCrossEntropy
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10
750/750 [==============================] - 9s 9ms/step - loss: 0.6463 - accuracy: 0.7990 - val_loss: 0.2374 - val_accuracy: 0.9268
Epoch 2/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3996 - accuracy: 0.8791 - val_loss: 0.1947 - val_accuracy: 0.9441
Epoch 3/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3550 - accuracy: 0.8934 - val_loss: 0.1844 - val_accuracy: 0.9471
Epoch 4/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3323 - accuracy: 0.9007 - val_loss: 0.1741 - val_accuracy: 0.9495
Epoch 5/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3133 - accuracy: 0.9076 - val_loss: 0.1588 - val_accuracy: 0.9548
Epoch 6/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3029 - accuracy: 0.9093 - val_loss: 0.1501 - val_accuracy: 0.9577
Epoch 7/10
750/750 [==============================] - 7s 9ms/step - loss: 0.2859 - accuracy: 0.9138 - val_loss: 0.1449 - val_accuracy: 0.9583
Epoch 

In [28]:
model.predict(x_test[71:72])

array([[  5.304846 , -20.16086  ,  -6.09499  ,  -9.596409 , -15.074245 ,
         -8.132149 ,  -4.720481 , -11.903206 ,  -7.3998404,  -8.102588 ]],
      dtype=float32)

In [29]:
y_test[71:72]

array([0], dtype=uint8)

## Сохранение модели

 `model.save()` при вызове сохраняет модель в один файл.
Сохраненный файл включает в себя:
- архитектуру модели;
- веса модели;
- config-file (если такой был создан при компиляции);
- оптимизатор и его состояние.

In [ ]:
model.save("path_to_my_model")
del model
# Recreate the exact same model purely from the file:
model = keras.models.load_model("path_to_my_model")

INFO:tensorflow:Assets written to: path_to_my_model/assets


In [ ]:
model.summary()

Model: "our_first_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 32)           25120       input_5[0][0]                    
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 32)           25120       input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32)           128         dense_23[0][0]                   
____________________________________________________________________________________